In [1]:
# https://github.com/scipy/scipy/issues/7151
# https://apps.dtic.mil/sti/pdfs/AD1004183.pdf
# https://www.codeproject.com/Articles/21282/Compute-Permanent-of-a-Matrix-with-Ryser-s-Algorit

# https://rosettacode.org/wiki/Determinant_and_permanent
# https://codegolf.stackexchange.com/questions/97060/calculate-the-permanent-as-quickly-as-possible

# https://stackoverflow.com/questions/38738835/generating-gray-codes
# https://qiita.com/b1ueskydragon/items/75cfee42541ea723080c

# https://qiita.com/phdax/items/3064de264c7933bab2f5
# https://web.archive.org/web/20190108235115/https://www.hackersdelight.org/hdcodetxt/pop.c.txt
# http://graphics.stanford.edu/~seander/bithacks.html#CountBitsSetParallel
# https://stackoverflow.com/questions/9829578/fast-way-of-counting-non-zero-bits-in-positive-integer

# https://stackoverflow.com/questions/22227595/convert-integer-to-binary-array-with-suitable-padding

In [2]:
import numpy as np
import time
from numba import jit

In [30]:
# https://github.com/XanaduAI/thewalrus/blob/master/thewalrus/_permanent.py

@jit(nopython=True)
def perm_bbfg(M):  # pragma: no cover
    """
    Returns the permanent of a matrix using the bbfg formula in Gray ordering
    The code is a re-implementation from a Python 2 code found in
    `Permanent code golf
    <https://codegolf.stackexchange.com/questions/97060/calculate-the-permanent-as-quickly-as-possible>`_
    using Numba.
    Args:
        M (array) : a square array.
    Returns:
        float or complex: the permanent of a matrix ``M``
    """

    n = len(M)
    if n == 0:
        return M.dtype.type(1.0)
    row_comb = np.sum(M, 0)
    #print(row_comb)
    #print()
    total = 0
    old_gray = 0
    sign = +1
    binary_power_dict = np.array([2**i for i in range(n)])
    num_loops = 2 ** (n - 1)
    for bin_index in range(1, num_loops + 1):
        #print()
        #print("row_comb",row_comb)
        #print()
        reduced = np.prod(row_comb)
        #print()
        #print("reduced",reduced)
        #print()
        total += sign * reduced
        new_gray = bin_index ^ (bin_index // 2)
        gray_diff = old_gray ^ new_gray
        gray_diff_index = np.searchsorted(binary_power_dict,gray_diff)
        #print("gray_diff",gray_diff)
        #print("gray_diff_index",gray_diff_index)
        new_vector = M[gray_diff_index]
        #print()
        #print("new_vector",new_vector)
        #print()
        direction = 2 * ((old_gray > new_gray) - (old_gray < new_gray))
        #print("direction",direction)
        #print()
        for i in range(n):
            row_comb[i] += new_vector[i] * direction
        sign = -sign
        old_gray = new_gray
    return total / num_loops

In [38]:
#L = 28
L = 30
a = np.ones((L,L),dtype=np.float64)
#a = np.zeros((L,L),dtype=np.float64)
#for i in range(L):
#    for j in range(L):
#        a[i,j] = i*L+j+1
#print(a)
print("L:",L)
start = time.time()
print(perm_bbfg(a))
end = time.time()
print("time:",end-start)
print("")

L: 30
2.6525286056257648e+32
time: 25.711397171020508



In [33]:
# for L in range(4,33):
#     a = np.ones((L,L),dtype=np.float64)
#     #print(a)
#     print("L:",L)
#     start = time.time()
#     print(perm_bbfg(a))
#     end = time.time()
#     print("time:",end-start)
#     print("")